In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = GermanDataset(device=device)

In [3]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_fb_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_fb_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_fb_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_fb_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_fb_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

# SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 256
SGD_hp_test['fairness'] = ''
SGD_hp_test['delta_effort'] = 1.1

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 89.05epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 163.50epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 165.80epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 158.05epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ----------  ---------
         0.0001          0          0.36875          0.3875   0.0171721  0.025     0.00454437  0.0765957
         0.001           0          0.7              0.73125  0          0.01875   0.0083712   0.0202128
         0.01            0          0.764062         0.74375  0.0357143  0.01875   0.0162641   0.138298
         0.1             0          0.767188         0.7375   0          0.009375  0.0251136   0.0882979


In [5]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.1
SGD_hp['lambda_'] = 0

# EI Hyperparameter Selection

In [6]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [1e-3, 1e-2, 1e-1]
EI_hp_test['lambda_'] = [1e-4, 1e-3, 1e-2, 0.1, 0.2]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 116.52epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 104.38epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 101.21epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 96.66epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ----------  ---------
          0.001     0.0001          0.698438         0.73125  0          0.01875   0.0083712   0.0202128
          0.001     0.001           0.698438         0.73125  0          0.01875   0.0083712   0.0202128
          0.001     0.01            0.698438         0.73125  0          0.01875   0.0083712   0.0202128
          0.001     0.1             0.7              0.71875  0          0.009375  0.0176991   0.0351064
          0.001     0.2             0.70625          0.70625  0          0.009375  0.0176991   0.0176991
          0.01      0.0001          0.7625           0.74375  0.0357143  0.01875   0.0162641   0.138298
          0.01      0.001           0.7625           0.75     0.037037   0.0125    0.00741449  0.138298
          0.01      0.01

In [7]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.5

# DP Hyperparameter Selection

In [8]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [1e-3, 1e-2]
DP_hp_test['lambda_'] = [1e-3, 1e-2, 0.1, 0.2, 0.5, 0.8]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 131.59epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 140.71epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 129.62epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 141.38epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei        dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  --------  ---------  ---------
          0.001      0.001          0.7              0.73125  0           0.01875   0.0083712  0.0202128
          0.001      0.01           0.701562         0.73125  0.0833333   0.01875   0.0083712  0.0202128
          0.001      0.1            0.6875           0.7375   0.111111    0.065625  0.064817   0.064817
          0.001      0.2            0.670312         0.725    0.0518039   0.065625  0.07558    0.07558
          0.001      0.5            0.592187         0.65     0.00466418  0.08125   0.0961492  0.0961492
          0.001      0.8            0.503125         0.575    0.0104244   0.0375    0.0710356  0.0829787
          0.01       0.001          0.764062         0.74375  0.0357143   0.01875   0.0162641  0.138298
          0.01       0.01 

In [9]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.01

# EO Hyperparameter Selection

In [10]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [1e-3, 1e-2]
EO_hp_test['lambda_'] = [1e-3, 1e-2, 0.1, 0.2, 0.5, 0.8]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 124.91epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 129.78epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 131.00epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 124.15epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ---------  ---------
          0.001      0.001          0.7              0.73125  0          0.01875   0.0083712  0.0202128
          0.001      0.01           0.7              0.73125  0          0.01875   0.0083712  0.0202128
          0.001      0.1            0.701562         0.7375   0          0.009375  0.0265487  0.0265487
          0.001      0.2            0.704688         0.7125   0          0.0125    0.0265487  0.0265487
          0.001      0.5            0.7              0.725    0          0.0125    0          0.0138298
          0.001      0.8            0.7              0.70625  0          0         0          0
          0.01       0.001          0.764062         0.74375  0.0357143  0.01875   0.0162641  0.138298
          0.01       0.01           0.76

In [11]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.001
EO_hp['lambda_'] = 0.1

# EOD Hyperparameter Selection

In [12]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [1e-2, 5e-2, 1e-1]
EODD_hp_test['lambda_'] = [1e-3,1e-2,1e-1,.2,.5,.9]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 110.18epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 105.42epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 108.14epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 104.41epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ---------  ---------
           0.01      0.001          0.764062         0.74375  0.0357143  0.01875   0.0162641  0.138298
           0.01      0.01           0.7625           0.7375   0.0740741  0.028125  0.0162641  0.159574
           0.01      0.1            0.759375         0.725    0.0740741  0.04375   0.0251136  0.180851
           0.01      0.2            0.767188         0.74375  0.0833333  0.05625   0.052619   0.180851
           0.01      0.5            0.754687         0.7375   0          0.053125  0.0265487  0.179787
           0.01      0.9            0.7              0.70625  0          0         0          0
           0.05      0.001          0.764062         0.73125  0          0.021875  0.0162641  0.130851
           0.05      0.01           0.760938 

In [16]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.1
EODD_hp['lambda_'] = 0.5

# Model Training

In [17]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 150.50epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:00<00:00, 155.05epochs/s]


training seed 3 started


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:00<00:00, 109.31epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:00<00:00, 110.68epochs/s]

Training finished for all seeds.


In [20]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var     ei_mean     ei_var    dp_mean     dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ----------  ---------  ---------  ---------  ---------  ----------  -----------  ----------
SGD                0.778      0.00927362  0.0495286   0.025231   0.0796173  0.0138203  0.057201   0.0118742     0.0710606  0.0167167
EI                 0.752      0.0132665   0.00920984  0.0114004  0.0582593  0.0188254  0.058667   0.00790257    0.0591609  0.00742206
DP                 0.783      0.00979796  0.0442969   0.0139057  0.0772099  0.0100478  0.0606217  0.00955543    0.0663325  0.00756611
EO                 0.741      0.0297321   0.117275    0.104782   0.018642   0.0164306  0.0195466  0.0149159     0.0313805  0.0106673
EODD               0.734      0.0243721   0.0657927   0.0385516  0.0196914  0.0128214  0.0210669  0.0160285     0.0690435  0.0412486
